In [29]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key='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'
)

response=llm.invoke("what is the name of first person in space...")
print(response.content)

The name of the first person in space is Yuri Gagarin. He was a Soviet cosmonaut who made history by becoming the first human to journey into outer space on April 12, 1961. Gagarin orbited the Earth once aboard the Vostok 1 spacecraft, completing a single orbit in 108 minutes.


In [31]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-40715?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Software Engineer II, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer II, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc. doe

In [32]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template("""
               ### SCRAPED TEXT FROM WEBSITE: tpage_data) 
               ### INSTRUCTION: The scraped text is from the career's page of a website. 
               Your job is to extract the job postings and return them in JSON format contail following keys: role, experience, skills and description. Only return the valid JSON .
### VALID JSON (NO PREAMBLE):""")
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
[
  {
    "role": "Software Engineer",
    "experience": "3-5 years",
    "skills": ["Java", "Spring", "Hibernate"],
    "description": "We are looking for a skilled Software Engineer to join our team. The ideal candidate will have experience in Java, Spring, and Hibernate. Responsibilities include designing, developing, and testing software applications."
  },
  {
    "role": "Data Scientist",
    "experience": "5-7 years",
    "skills": ["Python", "Machine Learning", "Data Visualization"],
    "description": "We are seeking a highly skilled Data Scientist to join our team. The ideal candidate will have experience in Python, Machine Learning, and Data Visualization. Responsibilities include analyzing complex data sets and developing predictive models."
  },
  {
    "role": "DevOps Engineer",
    "experience": "2-4 years",
    "skills": ["AWS", "Docker", "Kubernetes"],
    "description": "We are looking for a skilled DevOps Engineer to join our team. The ideal candidate will ha

In [34]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Software Engineer',
  'experience': '3-5 years',
  'skills': ['Java', 'Spring', 'Hibernate'],
  'description': 'We are looking for a skilled Software Engineer to join our team. The ideal candidate will have experience in Java, Spring, and Hibernate. Responsibilities include designing, developing, and testing software applications.'},
 {'role': 'Data Scientist',
  'experience': '5-7 years',
  'skills': ['Python', 'Machine Learning', 'Data Visualization'],
  'description': 'We are seeking a highly skilled Data Scientist to join our team. The ideal candidate will have experience in Python, Machine Learning, and Data Visualization. Responsibilities include analyzing complex data sets and developing predictive models.'},
 {'role': 'DevOps Engineer',
  'experience': '2-4 years',
  'skills': ['AWS', 'Docker', 'Kubernetes'],
  'description': 'We are looking for a skilled DevOps Engineer to join our team. The ideal candidate will have experience in AWS, Docker, and Kubernetes. Respon

In [35]:
type(json_res)

list

In [36]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [38]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [39]:
links = collection.query(query_texts=["experience in python", "expertise in react native"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [43]:
job = json_res

In [45]:
all_skills = [skill for j in job for skill in j['skills']]
all_skills

['Java',
 'Spring',
 'Hibernate',
 'Python',
 'Machine Learning',
 'Data Visualization',
 'AWS',
 'Docker',
 'Kubernetes']

In [47]:
links = collection.query(query_texts=[skill for j in job for skill in j['skills']], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}]]

In [50]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Abhijat, a business development executive at I0I. I0I is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of I0I 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase I0I's portfolio: {link_list}
        Remember you are Abhijat, BDE at I0I. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Solutions for Your Software Engineer, Data Scientist, and DevOps Engineer Needs

Dear Hiring Manager,

I hope this email finds you well. I came across your job postings for a Software Engineer, Data Scientist, and DevOps Engineer, and I am excited to introduce you to I0I, an AI & Software Consulting company that can help you find the perfect candidates for these roles.

At I0I, we have a proven track record of empowering numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has extensive experience in designing, developing, and testing software applications, analyzing complex data sets, and developing predictive models, as well as designing and implementing scalable infrastructure solutions.

Our portfolio showcases our capabilities in:

* Java and Spring development: https://example.com/java-portfolio
* Machine Learning and Python development: https://example.com